<a href="https://colab.research.google.com/github/ZoliN/colab/blob/main/blade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/zip.zip" -d /content/
%cd /content/zip
!mkdir build
%cd build
!cmake .. -DCMAKE_BUILD_TYPE=Release

In [ ]:
%cd /content
!wget  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!mkdir pics
!unzip DIV2K_train_HR.zip -d pics/

In [ ]:
%cd /content/pics/DIV2K_train_HR
!sh /content/drive/MyDrive/blade/copypics.sh

/content/pics/DIV2K_train_HR


In [ ]:
 %cd /content/zip/build
!make -j4

/content/zip/build
[ 10%] Building CXX object CMakeFiles/blade.dir/HashBuckets.cpp.o
[ 20%] Linking CXX executable blade
[100%] Built target blade


In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/2023_5_28_1_35_27.filter"  /content/zip/filters/

In [ ]:
!pwd

In [ ]:
%cd /content/zip/train_images
!mv *.png ./p/
!mv ./p/08.png .
!for i in {1..1000}; do cp 08.png "test$i.png"; done

/content/zip/train_images


In [ ]:
%cd /content/zip/
!./build/blade
!cp -r filters /content/drive/MyDrive/blade/
!cp -r result_images /content/drive/MyDrive/blade/
!cp temp_images/*.csv /content/drive/MyDrive/blade/
while True:pass

In [ ]:
!zip -r /content/drive/MyDrive/bladearch/src.zip /content/drive/MyDrive/blade/*

In [ ]:
%cd /content/
!mkdir build
%cd build
!cmake /content/drive/MyDrive/superrestf/bladesr -DCMAKE_BUILD_TYPE=Release

In [ ]:
 %cd /content/build
!make -j4

In [ ]:
import os
os.makedirs('/content/anal_images', exist_ok=True)
!/content/build/blade

import os
import tensorflow as tf
import cv2

def iterate_files(directory):
    for file in os.listdir(directory):
        yield os.path.join(directory, file)

inpath = '/content/anal_images'
outpath = '/content/anal_tensors'
os.makedirs(outpath, exist_ok=True)
# Iterate over the files of the current directory
for file in iterate_files(inpath):
    filename = os.path.basename(file)
    print(filename)
    im_input = cv2.imread(file, -1)
    print(im_input.shape)

    byte_string = tf.io.serialize_tensor(im_input)
    # Write the byte string to a file
    with tf.io.gfile.GFile(outpath + '/' + filename[:-5] + '.bin', 'wb') as f:
        f.write(byte_string.numpy())


In [9]:
import os
import tensorflow as tf
import cv2
import numpy as np

analpath = '/content/anal_tensors'
byte_string = []
with tf.io.gfile.GFile(analpath + '/' + '12.png' + '.bin', 'rb') as f:
    byte_string = f.read()

# Deserialize the tensor from the byte string
anal_tensor = tf.io.parse_tensor(byte_string, out_type=tf.float32)

lam1, lam2, c, s = tf.unstack(anal_tensor, axis=-1)

Dtr = 3000. * 1.525902E-05 * 0.860704768
Dth = 1000. * 1.525902E-05 * 0.860704768 * 1. / Dtr;
kDetail = 0.5;
kDenoise = 1.55;
kStretch = 4.;
kShrink = 1.;

A = (lam1 - lam2) / (lam1 + lam2 + 0.0000000001);
A = tf.clip_by_value(A, 0.1, 1.0);

D = 1. - tf.sqrt(lam1) / Dtr + Dth
D = tf.clip_by_value(D, 0.0, 1.0)

k1h = kDetail * (kStretch * A +  (1. - A));
k2h = kDetail / (kShrink * A +  (1. - A));

k1 = ((1.0 - D)*k1h + D*kDenoise);
k2 = ((1.0 - D)*k2h + D*kDenoise);
k1 *= k1;
k2 *= k2;

x2 = c;
y2 = s;
x1 = s;
y1 = -c;

b11 = k1*x1*x1 + x2*x2*k2;
b12 = k1*x1*y1 + x2*y2*k2;
b22 = k1*y1*y1 + y2*y2*k2;

det = b11*b22 - b12*b12 + 0.0000000001;

m11 = b22 / det;
m22 = b11 / det;
m12 = -b12 / det;



img_raw = tf.io.read_file('/content/drive/MyDrive/superrestf/bladesr/test_images/12.png')
im_input = tf.image.decode_image(img_raw)
im_input = tf.cast(im_input, dtype=tf.float32)/255.0


scale = 2
scalef = float(scale)
invscale = 1./scalef

orig_sz = tf.shape(im_input)[0:2]
lr_sz = orig_sz // scale
full_sz = lr_sz * scale
im_lr = tf.image.resize(im_input, lr_sz, method='area')

im_hr = tf.image.resize(im_lr, full_sz, method='nearest')

cv2.imwrite('/content/im_hr.png', im_hr.numpy()*255.0)

im_hr2 = tf.image.resize(im_input, full_sz, method='lanczos5')
cv2.imwrite('/content/im_hr2.png', im_hr2.numpy()*255.0)

kerp = tf.stack((m11, m22, m12), axis = -1)
kerp = tf.image.resize(kerp, full_sz, method='bilinear')
m11, m22, m12 = tf.split(kerp, 3, axis = -1)

#paddings = tf.constant([[2 * scale, 2 * scale], [2 * scale, 2 * scale], [0, 0]])
paddings = tf.constant([[2, 2], [2, 2], [0, 0]])
im_padded = tf.pad(im_hr, paddings, "SYMMETRIC")

#tf.convert_to_tensor(value, dtype=None

im_dst = tf.fill(tf.shape(im_hr), 0.0)
im_w = tf.fill(tf.shape(im_hr), 0.0)


print(tf.shape(input=im_dst))
print(tf.shape(input=im_padded))


# Create two 1D arrays
x_coords = np.arange(float(full_sz[0]))
y_coords = np.arange(float(full_sz[1]))

# Create a 2D NumPy array containing x and y indices
indices = np.meshgrid(x_coords, y_coords)

srcpos1 = np.stack([indices[0], indices[1]], 2)
srcpos1 = (srcpos1 + 0.4995) * invscale
srcpos = tf.convert_to_tensor(srcpos1, dtype=tf.float32)
srcMidPos = tf.floor(srcpos) + 0.5;

# for y in range(5):
#   ys = y * scale
#   y1 = float(y - 2)
#   for x in range(5):
#     xs = x * scale
#     x1 = float(x - 2)
#     posk = (srcMidPos + (x1, y1) - srcpos) * scalef
#     poskx, posky = tf.split(posk, 2, axis = -1)
#     #if (x==2 and y==2):
#     #  print(poskx.numpy()[0:4,0:4])
#     kw = tf.exp(-0.5 * ((poskx*m11+posky*m12)*poskx+(poskx*m12+posky*m22)*posky) )
#     #if (x==2 and y==2):
#     #  print(kw.numpy()[0:4,0:4])
#     im_w += kw
#     im_dst+= kw * im_padded[ys:ys+full_sz[0], xs: xs+full_sz[1]]

halfRatio = invscale * 0.5

for y in range(5):
  y1 = float(y - 2)
  for x in range(5):
    x1 = float(x - 2)
    srcposc = srcpos + (x1 * invscale, y1 * invscale)
    comp = tf.where(tf.abs(srcposc - tf.floor(srcposc) - 0.5) < halfRatio, 1.0, 0.0)
    compx, compy = tf.split(comp, 2, axis = -1)
    kw = tf.where(compx * compx == 1.0, tf.exp(-0.5 * ((x1*m11+y1*m12)*x1+(x1*m12+y1*m22)*y1) ), 0.0)
    if (x==3 and y==2):
      print(kw.numpy()[200:204,200:204])
    im_w += kw
    im_dst+= kw * im_padded[y:y+full_sz[0], x: x+full_sz[1]]


im_dst = im_dst / (im_w + 0.0000001)

cv2.imwrite('/content/out.png', im_dst.numpy()*255.0)

cv2.imwrite('/content/test.jpg', A.numpy()*255.0)

print(tf.shape(input=D))

tf.Tensor([512 512   1], shape=(3,), dtype=int32)
tf.Tensor([516 516   1], shape=(3,), dtype=int32)
[[[0.13539046]
  [0.        ]
  [0.13534747]
  [0.        ]]

 [[0.13547847]
  [0.        ]
  [0.13563393]
  [0.        ]]

 [[0.1357042 ]
  [0.        ]
  [0.1362247 ]
  [0.        ]]

 [[0.13593206]
  [0.        ]
  [0.13677435]
  [0.        ]]]
tf.Tensor([256 256], shape=(2,), dtype=int32)


In [ ]:
import numpy as np



# Create two 1D arrays
x_coords = np.arange(3.)
y_coords = np.arange(4.)

# Create a 2D NumPy array containing x and y indices
indices = np.meshgrid(x_coords, y_coords)

coords = np.stack([indices[0], indices[1]],2)
coords += 0.4995

# Print the first element of the array
print(coords)  # Output: (0.0, 0.0)

[[[0.4995 0.4995]
  [1.4995 0.4995]
  [2.4995 0.4995]]

 [[0.4995 1.4995]
  [1.4995 1.4995]
  [2.4995 1.4995]]

 [[0.4995 2.4995]
  [1.4995 2.4995]
  [2.4995 2.4995]]

 [[0.4995 3.4995]
  [1.4995 3.4995]
  [2.4995 3.4995]]]
